In [1]:
import pandas as pd
import json

from pathlib import Path
import sys
import os

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.config import config
from src.data.etl import load_data
from src.model.bert import TextEmbedder, DimensionalityReducer, ClusteringModel, CTFIDFVectorizer
from src.model.eval import TopicModelEvaluator
from src.model.label import LLMTopicNamer

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
print(config)

Settings Configuration:
  BASE_PATH: /Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling
  DATA_FILE_PATH: /Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/data/raw/data.json
  SEED: 23
  LLM_CONFIG: {'model_name': 'Qwen/Qwen3-4B'}
  LOG_CONFIG: {'version': 1, 'disable_existing_loggers': False, 'formatters': {'detailed': {'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s', 'datefmt': '%Y-%m-%d %H:%M:%S'}, 'simple': {'format': '%(levelname)s - %(message)s'}}, 'handlers': {'console': {'class': 'logging.StreamHandler', 'level': 'INFO', 'formatter': 'simple', 'stream': 'ext://sys.stdout'}, 'file': {'class': 'logging.handlers.RotatingFileHandler', 'level': 'DEBUG', 'formatter': 'detailed', 'filename': '/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/output/topic_modeling.log', 'maxBytes': 10485760, 'backupCount': 5, 'encoding': 'utf-8'}}, 'root': {'level': 'INFO', 'handlers': ['console', 'file']}}
  embedding_model_config: {'model_name': 'sentence-tran

In [3]:
data = load_data()

data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [4]:
docs = data['reviewText'].tolist()

print("Number of Docs:",len(docs))

print("Unique ASINs:",data['asin'].nunique())
print("Sample documents:")
docs[:8]

Number of Docs: 194439
Unique ASINs: 10429
Sample documents:


["They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again",
 'These stickers work like the review says they do. They stick on great and they stay on the phone. They are super stylish and I can share them with my sister. :)',
 'These are awesome and make my phone look so stylish! I have only used one so far and have had it on for almost a year! CAN YOU BELIEVE THAT! ONE YEAR!! Great quality!',
 "Item arrived in great time and was in perfect condition. However, I ordered these buttons because they were a great deal and included a FREE screen protector. I never received one. Though its not a big deal, it would've been nice to get it since they claim it comes with one.",
 'awesome! stays on, and looks great. can be used on multiple apple products.  especially having nails, it helps to have an elevated key.',
 'These make using the home button easy. My daught

In [ ]:
embedder = TextEmbedder(**config.embedding_model_config)
embeddings = embedder.embed(docs)

print("Embeddings shape:", embeddings.shape)

INFO - Use pytorch device_name: mps
INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Using device: Apple Silicon GPU (MPS)
  macOS: True


Batches: 100%|██████████| 6077/6077 [01:57<00:00, 51.84it/s] 


Embeddings shape: (194439, 384)


In [ ]:
reducer = DimensionalityReducer(**config.dr_config)
reduced_embeddings = reducer.fit_transform(embeddings)

INFO - Initialized UMAP with n_components=5


AttributeError: 'DimensionalityReducer' object has no attribute 'reduce'